In [1]:
from dotenv import load_dotenv


In [2]:
load_dotenv()

True

# Local MCP server

In [3]:
from langchain_mcp_adapters.client import MultiServerMCPClient

client = MultiServerMCPClient(
    {
        "local_server": {
            "transport": 'stdio',
            "command": "python",
            "args": ["resources/1_mcp_server.py"]
        }
    }
)

In [4]:
# get tools
tools = await client.get_tools()

# get resources 
resources = await client.get_resources("local_server")

# get prompts 
prompt = await client.get_prompt("local_server", "prompt")
prompt = prompt[0].content

In [5]:
from langchain.chat_models import init_chat_model
from langchain.agents import create_agent

In [6]:
model = init_chat_model(model="gemini-2.5-flash", model_provider="google_genai")
agent = create_agent(
    model = model,
    tools=tools,
    system_prompt = prompt,
)

In [7]:
from langchain.messages import HumanMessage

config = {"configurable": {"thread_id": "1"}}

response = await agent.ainvoke(
    {"messages": [HumanMessage(content="Tell me about the langchain-mcp-adapters library")]},
    config=config
)

In [8]:
from pprint import pprint

pprint(response)

{'messages': [HumanMessage(content='Tell me about the langchain-mcp-adapters library', additional_kwargs={}, response_metadata={}, id='5fe0be9f-1f74-4b23-b504-3f2aedaf3cb0'),
              AIMessage(content='', additional_kwargs={'function_call': {'name': 'search_web', 'arguments': '{"query": "langchain-mcp-adapters library"}'}, '__gemini_function_call_thought_signatures__': {'ccb9c664-02b0-4eeb-b8f0-395ce2c66499': 'CtEBAXLI2nzuChb41cS7PcYDWxKB1LPAXf6LdEez3ApRBniSotwH1o+2GF932oEuIF4n2OvEOG4NT9FDhuwPS6l5jTlAdqCQsgAuXgfqJiBwbTKiCYksfyGLuumiKNKQxTUhbtXZDbcRHaifj/+jngCEx7yWrNP8W3chLMZp+4SH0w3EseZCB5DdHzjSAVlUOmzR2WPg4h10NqlDhIceMKFqTaqk/4Y4GquLevwn1z3Y60x3j80IQEUybKZwQ7fB4LMlAY2DRVvES8dJn22tLys1wxU='}}, response_metadata={'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': [], 'model_provider': 'google_genai'}, id='lc_run--019bc7c3-bd4b-7b81-8ba5-81e1a4de11ab-0', tool_calls=[{'name': 'search_web', 'args': {'query': 'langchain-mcp-adapters library'}, 'id': 'ccb9c664

# Online MCP

In [9]:
client = MultiServerMCPClient(
    {
        "time": {
            "transport": "stdio",
            "command": "uvx",
            "args": [
                "mcp-server-time",
                "--local-timezone=America/New_York"
            ]
        }
    }
)

tools = await client.get_tools()

In [10]:
agent = create_agent(
    model=model,
    tools=tools
)

In [11]:
question = HumanMessage(content="What time is it?")

response = await agent.ainvoke(
    {"messages": [question]}
)

print(response)

{'messages': [HumanMessage(content='What time is it?', additional_kwargs={}, response_metadata={}, id='4d9650fe-8cc2-41a0-bd68-8a8b982dfdf9'), AIMessage(content='', additional_kwargs={'function_call': {'name': 'get_current_time', 'arguments': '{"timezone": "America/New_York"}'}, '__gemini_function_call_thought_signatures__': {'7ac4ce10-8876-47ff-bb70-25fc4ba85be0': 'Cu4CAXLI2nwKuHf1/ZeV/4TV+ASbOSr85WmHdO0L3KJoLvL8hn1mfUE4SB6dLhZUTidebZv/6vjbDgOZefyn0mlSDPpLLqBoqIvcRixLtkobL7pELXw4pZM6iFR9JiXpONxjJLUvStAjIOMfmJ7UT59KL2RsBdZ3pl21ODS4rxuu2ioCGPSNLYsROnqvcK58js5XHTvRf3Try0iYt2qllJ4i94J0FfnyXTurnTJDUlmpcLIe/KeK39XmkG2GpEdVLAst6gvHuJ1JxVY7VG6t9tPa1IkJBlq2tdadzCuYGYDPh20nwxVeG935nUPBYCFxkNbTej9unNqU2SFRM/nHmO4ozeDN6jD7WRBy27CN8MsbaYEiufLhOSdv1JyQQnR2m1jkpbNrOFFhZTHgGKIlbKcHGvDiHjO+UG3v+jdAQou7LdVAEkJsQO3WExoUV1iqSjNSLSoCsz7dFDpcNZ07w+L860mGsEqAUP/W3yk2s8Ve'}}, response_metadata={'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': [], 'model_provider': 'google_genai'},

In [12]:
response["messages"][-1].content

'The current time in America/New_York is 12:04:37 on Friday, January 16, 2026.'